In [1]:
def MNIST_Bootstrap_Exp(Num_Tasks,Num_Epochs):
    %matplotlib notebook
    import keras as keras
    from keras.models import Sequential
    from keras.layers import LSTM, Dense,BatchNormalization,Dropout,Flatten, Conv1D
    from keras.losses import binary_crossentropy, categorical_crossentropy
    from keras.metrics import categorical_accuracy
    from keras import regularizers,optimizers
    from keras.regularizers import l2
    import numpy as np
    import scipy.io
    import matplotlib.pyplot as plt
    import gzip
    from keras.layers.core import Lambda
    from scipy.integrate import trapz
    import seaborn as sns

    from keras.layers.core import Lambda
    from keras import backend as K    


    def mini_batches(InputSample,BatchSize):
        Index = np.array(range(InputSample.shape[0]),dtype=int)
        NumBatches = np.int(InputSample.shape[0]/BatchSize)
        Removed = np.array([],dtype=int)

        BatchInd =[]
        for BatchLoop in range(NumBatches):
            RemainIndex = np.delete(Index,Removed)
            SampleInd = np.random.choice(RemainIndex,size=BatchSize,replace=False)
            Removed = np.append(Removed,SampleInd,axis=0)

            BatchInd.append(SampleInd)
        RemainIndex = np.delete(Index,Removed)
        BatchInd.append(RemainIndex)

        return BatchInd,NumBatches

    def Get_Feats_and_Targets(filename):
        import numpy as np

        def line_to_Feats(line):
            line = line.split(' ')
            Feats = np.asarray(line[0:1024])
            Target = np.zeros([3])
            Target[int(line[1024])] = 1
            return Feats,Target

        f = open(filename, 'r')
        lines = f.readlines()
        Features = []
        Targets = []
        for i in range(len(lines)-1):
            line = lines[i+1]
            Feats,Tgts = line_to_Feats(line)
            Features.append(Feats)
            Targets.append(Tgts)

        return np.asarray(Features,dtype = 'float64'), np.asarray(Targets,dtype = 'int')

    FullInputs = scipy.io.loadmat('MNIST_TrainInputs.mat')
    FullInputs = FullInputs['images']

    FullTargets = scipy.io.loadmat('MNIST_TrainTargets.mat')
    FullTargets = FullTargets['targets']

    Validation_Cutoff = 0.75

    Validation_Cutoff = np.int(Validation_Cutoff*FullInputs.shape[0])

    ValInputs = FullInputs[Validation_Cutoff:,:]
    ValTargets = FullTargets[Validation_Cutoff:,:]

    TrainInputs = FullInputs[0:Validation_Cutoff,:]
    TrainTargets = FullTargets[0:Validation_Cutoff,:]

    data_dim = TrainInputs.shape[-1]
    NumSamples = TrainInputs.shape[0]
    Num_Targets = TrainTargets.shape[-1]

    index = np.linspace(0,NumSamples,NumSamples,endpoint=False,dtype=int)

    reg_coeff = 0.001

    def Gen_Model(reg_coeff):
        model = Sequential()
        model.add(Dense(100,activation='relu',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
        model.add(Dropout(0.25))
        model.add(Dense(100,activation='relu',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
        model.add(Dropout(0.25))
        model.add(Dense(100,activation='relu',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
        model.add(Dropout(0.25))
        model.add(Dense(Num_Targets,activation = 'softmax',kernel_regularizer=l2(reg_coeff),input_shape =(data_dim,)))
        optim = optimizers.adam(lr=0.0001)
        model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['categorical_accuracy'])
        return model

    D2THard_model = Gen_Model(reg_coeff)
    D2TEasy_model = Gen_Model(reg_coeff)
    BALDHard_model = Gen_Model(reg_coeff)
    BALDEasy_model = Gen_Model(reg_coeff)

    Uni_model = Gen_Model(reg_coeff)

    D2THard_model.set_weights(Uni_model.get_weights())
    D2TEasy_model.set_weights(Uni_model.get_weights())
    BALDHard_model.set_weights(Uni_model.get_weights())
    BALDEasy_model.set_weights(Uni_model.get_weights())
    
    def acquisition_function_BALD(model,samples,Num_Targets,temperature=1,Target_Ratio = 5):
        nb_MC_samples = 100
        MC_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[-1].output])
        MC_samples = np.zeros([nb_MC_samples,samples.shape[0],Num_Targets])
        learning_phase = True 
        for i in range(nb_MC_samples):
            MC_samples[i,:,:] = np.array([MC_output([samples, learning_phase])[0]])

        expected_entropy = - np.mean(np.sum(MC_samples * np.log(MC_samples + 1e-10), axis=-1), axis=0)  # [batch size]
        expected_p = np.mean(MC_samples, axis=0)
        entropy_expected_p = - np.sum(expected_p * np.log(expected_p + 1e-10), axis=-1)  # [batch size]
        BALD_acq = entropy_expected_p - expected_entropy

        Exp_BALD = np.exp(BALD_acq/temperature)
        Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)

        Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
        Target_Ratio = Target_Ratio
        if Max_Prob_Ratio < Target_Ratio:
            while Max_Prob_Ratio <Target_Ratio:
                temperature = temperature*0.99
                Exp_BALD = np.exp(BALD_acq/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        else:
            while Max_Prob_Ratio > Target_Ratio:
                temperature = temperature*1.01
                Exp_BALD = np.exp(BALD_acq/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        return Sampling_Prob

    def acquisition_function_dist_to_threshold(model,samples,Num_Targets,temperature=1,Target_Ratio=5):
        Output = model.predict(samples)
        Output -= 1/float(Num_Targets)
        Dist_to_Threshold = np.sum(np.abs(Output),1)
        Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
        Exp_Dist_to_Threshold *= 1
    #     Exp_Dist_to_Threshold = Dist_to_Threshold
        Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
        Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
        Target_Ratio = Target_Ratio
        if Max_Prob_Ratio < Target_Ratio:
            while Max_Prob_Ratio <Target_Ratio:
                temperature = temperature*0.99
                Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        else:
            while Max_Prob_Ratio > Target_Ratio:
                temperature = temperature*1.01
                Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        return Sampling_Prob

    count = 0 
    Batch_Size = 50
    Uni_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=Num_Epochs,verbose=1,validation_data=[ValInputs,ValTargets])

    Dist_to_Threshold = acquisition_function_dist_to_threshold(Uni_model,TrainInputs,Num_Targets,1,10)

    Easy_Dist_to_Threshold_ArgSort = np.flipud(Dist_to_Threshold.argsort())
    Hard_Dist_to_Threshold_ArgSort = Dist_to_Threshold.argsort()

    NumTasks = Num_Tasks
    TaskSplitPoints = np.int(np.ceil(NumSamples/NumTasks))
    print(TaskSplitPoints)
    for i in range(NumTasks):
        Num_Epochs_Task = Num_Epochs*(1/(i+1))

        TaskInd = Hard_Dist_to_Threshold_ArgSort[0:np.min([(i+1)*TaskSplitPoints,NumSamples])]
        D2THard_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))

        TaskInd = Easy_Dist_to_Threshold_ArgSort[0:np.min([(i+1)*TaskSplitPoints,NumSamples])]
        D2TEasy_model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))

    TestInputs = scipy.io.loadmat('MNIST_TestInputs.mat')
    TestInputs = TestInputs['test_images']

    TestTargets = scipy.io.loadmat('MNIST_TestTargets.mat')
    TestTargets = TestTargets['test_targets']

    TestError = np.zeros([3,2])
    TestError[0,:] = D2THard_model.test_on_batch(TestInputs,TestTargets)
    TestError[1,:] = D2TEasy_model.test_on_batch(TestInputs,TestTargets)
    TestError[2,:] = Uni_model.test_on_batch(TestInputs,TestTargets)
    
    return TestError

In [ ]:
import numpy as np
NumTests = 100
from IPython.display import clear_output

TestError_Record = np.zeros([3,2,NumTests])

for i in range(NumTests):
    
    clear_output()
    print('Run:')
    print(i)
    TestError_Record[:,:,i] = MNIST_Bootstrap_Exp(5,100)
    np.save('MNIST_Bootstrap_Perf_5',TestError_Record)

Run:
11
Train on 45000 samples, validate on 15000 samples
Epoch 1/100
31050/45000 [===================>..........] - ETA: 2s - loss: 1.8271 - categorical_accuracy: 0.5020